# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [2]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'ht

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [7]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [9]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    # Provide multiple example responses (multi-shot)
    user_prompt += "\n\nBelow are several examples of how you might respond. "
    user_prompt += "Use similar formatting to provide your own final answer.\n"

    user_prompt += """
Example 1:
{
    "links": [
        {"type": "about page", "url": "https://example.com/about"},
        {"type": "careers page", "url": "https://example.com/careers"}
    ]
}

Example 2:
{
    "links": [
        {"type": "company history", "url": "https://example.org/company/history"},
        {"type": "investor relations", "url": "https://example.org/investors"}
    ]
}

Example 3:
{
    "links": [
        {"type": "products overview", "url": "https://another-example.com/products"},
        {"type": "blog", "url": "https://another-example.com/blog"}
    ]
}
"""

    user_prompt += "\nPlease format your response similarly, focusing only on relevant links for a company brochure."
    return user_prompt

In [10]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [11]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [12]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/meta-llama/Llama-4-Scout-17B-16E-Instruct',
 '/reducto/RolmOCR',
 '/agentica-org/DeepCoder-14B-Preview',
 '/meta-llama/Llama-4-Maverick-17B-128E-Instruct',
 '/deepseek-ai/DeepSeek-V3-0324',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/jamesliu1217/EasyControl_Ghibli',
 '/spaces/VAST-AI/TripoSG',
 '/spaces/Stable-X/Hi3DGen',
 '/spaces/victor/deepsite-gallery',
 '/spaces',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/open-thoughts/OpenThoughts2-1M',
 '/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset',
 '/datasets/agentica-org/DeepCoder-Preview-Dataset',
 '/datasets/LLM360/MegaMath',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel'

In [13]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [15]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)    
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [16]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company social media', 'url': 'https://twitter.com/huggingface'}, {'type': 'company social media', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
meta-llama/Llama-4-Scout-17B-16E-Instruct
Updated
1 day ago
•
228k
•
703
reducto/RolmOCR
Updated
8 days ago
•
3.15k
•
287
agentica-org/DeepCoder-14B-Preview
Updated
about 22 hours ago
•
3.35k
•
285
meta-llama/Llama-4-Mave

In [17]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [18]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [19]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'company homepage', 'url': 'https://huggingface.co'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation', 'url': 'https://huggingface.co/docs'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'community discussions', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub repository', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmeta-llama/Llama-4-Scout-17B-16E-Instruct\nUpdated\n1 day ago\n•\n228k\n•\n703\nreducto/RolmOCR\nUpdated\n8 days ago\n•\n3.15k\n•\n287\nagentica-org/DeepCoder-14B-Preview\nUpdated\nabout 22 hours ago\n•\n3.35k\n•\n285\nmeta-llama/Llama-4-Maverick-17B-128E-Instruct\nUpdated\n1 day ago\n•\n21k\n•\n271\ndeepseek-ai/DeepSeek-V3-0324\nUpdated\n15 days ago\n•\n183k\n•\n2.51k\nBrowse 1M+ models\nSpac

In [56]:
translate_system_prompt = "You are a professional translation assistant that takes any English brochure content provided and accurately translates it into Spanish while preserving the text's original tone, structure, formatting, headings, subheadings, and key terminologies, ensuring an easy-to-read, faithful rendering of the brochure."
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    response2 = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": translate_system_prompt},
            {"role": "user", "content": response.choices[0].message.content}
          ],
    )
    result = response2.choices[0].message.content
    display(Markdown(result))

In [58]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise solutions', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Folleto de Hugging Face

---

### Bienvenido a Hugging Face
**La comunidad de IA que está construyendo el futuro.**  
En Hugging Face, creamos una plataforma colaborativa para que la comunidad de aprendizaje automático (ML) explore, desarrolle y despliegue aplicaciones, conjuntos de datos y modelos de IA. ¡Únete a nosotros para liderar la próxima ola de innovación en IA!

---

### Nuestra Plataforma
- **Explora Aplicaciones de IA:** Descubre aplicaciones y soluciones de vanguardia. 
- **Modelos y Conjuntos de Datos:** Accede a más de 1 millón de modelos y más de 250,000 conjuntos de datos, incluidos modelos en tendencia en diversos dominios. 
- **Espacios:** Colabora y ejecuta aplicaciones en vivo creadas por nuestra comunidad.

### Servicios Ofrecidos
- **Soluciones de Cómputo:** Despliega fácilmente puntos de inferencia optimizados o actualiza tus aplicaciones a GPU en solo unos pocos clics, comenzando desde solo $0.60/hora.
- **Soluciones Empresariales:** Diseñadas para organizaciones. Disfruta de seguridad de nivel empresarial, colaboración en equipo y soporte dedicado, comenzando desde $20/usuario/mes.

---

### Nuestra Comunidad
Con más de **50,000 organizaciones** confiando en nosotros, incluidas gigantes como Google, Amazon y Microsoft, Hugging Face es el epicentro de los avances en IA. Nuestras bibliotecas de código abierto, como **Transformers** y **Diffusers**, empoderan a desarrolladores e investigadores para crear soluciones de ML de vanguardia.

---

### Cultura de la Empresa
En Hugging Face, nuestra cultura se define por **colaboración, innovación y comunidad**. Estamos comprometidos a construir herramientas y servicios inclusivos que todos puedan aprovechar. Nuestro equipo prospera al superar límites y creemos en compartir conocimientos para apoyarnos mutuamente en el campo en constante evolución de la IA.

---

### Oportunidades de Carrera
¡Únete a nuestra misión! Siempre estamos buscando personas apasionadas para unirse a nuestro equipo. Ya seas ingeniero, científico de datos o un innovador en IA, hay un lugar para ti en Hugging Face. Explora nuestras **[vacantes actuales](#)** y encuentra tu próxima aventura.

---

### Conéctate con Nosotros
¡Mantente conectado y únete a la conversación! Síguenos en:
- **[GitHub](#)**
- **[Twitter](#)**
- **[LinkedIn](#)**
- **[Discord](#)**

---

### ¡Comienza Hoy!
Sumérgete en el futuro de la IA con Hugging Face. Ya sea que busques explorar modelos, buscar soluciones empresariales o unirte a nuestro equipo dinámico, tenemos algo para todos.  
**[Regístrate](#)** para comenzar tu viaje!

---

**Hugging Face - Empoderando a la comunidad de IA, un modelo a la vez.**

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [23]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [24]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise solutions', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face Company Brochure

## About Us
**Hugging Face** is the leading collaborative platform for machine learning (ML) professionals, dedicated to building the future of AI technology. With a vibrant community, we believe in open-source, partnership, and sharing of knowledge to accelerate ML innovation.

### Our Mission
*To provide an accessible ecosystem that empowers machine learning practitioners to create, collaborate, and deploy models effectively.*

## What We Offer
- **Models**: Access over 1 million pre-trained models spanning various applications, including text, image, video, audio, and 3D.
- **Datasets**: Explore and collaborate with over 250,000 datasets designed for diverse machine learning tasks.
- **Spaces**: A platform to host and visualize applications with ease, empowering you to build and share AI-powered projects.
- **Enterprise Solutions**: Tailored solutions including dedicated support, security, and access controls, starting at $20/user/month.

## Our Customers
More than **50,000 organizations** trust Hugging Face, including major enterprises like:
- **Google**
- **Microsoft**
- **Amazon**
- **Meta**
- **Intel**

Our platform supports businesses wishing to harness AI technologies in innovative ways.

## Community and Culture
At Hugging Face, we prioritize a collaborative and inclusive community, where:
- **Innovators** come together to share models, methodologies, and thoughts.
- **All contributions** are valued, ensuring every voice is heard in shaping the platform’s evolution.

We embody a culture of continuous learning and open knowledge sharing to foster an environment where everyone can thrive together.

## Careers at Hugging Face
Be a part of our exciting journey! We are looking for passionate professionals to join our team and take part in shaping AI’s future. Our current openings range across various technical and non-technical roles including developing new features, enhancing platform capabilities, and engaging with our community.

### Why Work with Us?
- **Innovative Environment**: Work with cutting-edge technologies in machine learning.
- **Remote Flexibility**: Embrace the ability to work from anywhere.
- **Impactful Projects**: Contribute to initiatives that shape the direction of AI development globally.

## Get Started
Join the community and start collaborating today! Sign up on [Hugging Face](https://huggingface.co/join) and unlock the potential of AI and machine learning.

For more information, explore our [documentation](https://huggingface.co/docs) and check out our [latest posts](https://huggingface.co/posts) in the community. 

---

#### Contact Us
- **Website**: [huggingface.co](https://huggingface.co)
- **Social Media**: Connect with us on GitHub, Twitter, LinkedIn, and Discord.

Together, let’s build the future of AI!

In [25]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}



# Welcome to Hugging Face

**The AI community building the future.**

Hugging Face is a collaborative platform for the machine learning community. We provide tools to build, share, and collaborate on models, datasets, and applications, empowering innovators and organizations worldwide.

## Company Culture

At Hugging Face, we are dedicated to democratizing machine learning through open collaboration. Our culture fosters innovation, inclusivity, and a passion for AI. We believe in the power of community—more than 50,000 organizations rely on our platform to accelerate their AI projects. Join us as we create diverse solutions that push the boundaries of technology.

## Our Offerings

- **Models:** Explore over **1 million models** spanning various modalities including text, image, video, and audio. Whether you want state-of-the-art transformers or specialized datasets, we have got you covered.
- **Datasets:** Access and share **250,000+ datasets** optimized for various machine learning tasks.
- **Applications:** Collaborate with tools like **Spaces** to deploy apps seamlessly. From generating 3D models to building AI-driven applications, our platform supports your every need.
- **Enterprise Solutions:** Tailored services offering security and dedicated support, starting at **$20/user/month**.

## Who We Work With

Our community is vast, including industry leaders such as:
- **Google**
- **Microsoft**
- **Amazon**
- **Meta**

We empower enterprises and individuals alike to leverage advanced AI solutions that shape the future.

## Careers at Hugging Face

Join us in our mission! We're looking for passionate individuals to help advance the field of machine learning. Whether you are an engineer, researcher, or community advocate, explore our open roles on our [Jobs page](https://huggingface.co/jobs).

## Connect with Us

- **Twitter**
- **LinkedIn**
- **GitHub**
- **Discord**

Let’s build the future of AI together. Sign up today to start your journey with Hugging Face!

---

For more information, visit our website at [huggingface.co](https://huggingface.co).


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>